In [1]:
# import functions from ../definitions/definitions_EDA
from datetime import datetime, timedelta
import sys
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np
sys.path.append('../definitions')
import definitions_EDA as eda
# import definitions_plotting as def_plot
from scipy.fft import fft, ifft, fftfreq
from scipy import signal
import shutil
import seaborn as sns
from statsmodels.tsa.seasonal import seasonal_decompose
from dateutil.parser import parse
from pandas.plotting import autocorrelation_plot


Define Variables

In [2]:
start_date = '2024-03-20'
end_date = '2024-03-21'

Fetch data and store as dataframe (data ingestion)

In [3]:
data_arr_mag = eda.process_data(eda.get_data('ctumag', eda.read_txt_file, start_date, end_date))
data_arr_squid = eda.process_data(eda.get_data('squid', eda.read_txt_file, start_date, end_date))
df = eda.create_dataframe(data_arr_mag, data_arr_squid, start_date)
print(f' Head of dataframe: \n',df.head())
print(f' \n Shape of df', df.shape)
print(f'\nTypes for each column: \n', df.dtypes)

File not found: /Users/tristan/Library/CloudStorage/OneDrive-StellenboschUniversity/Academics/Final_year/Semester 2/Skripsie/Data/DUMMY/2024-03-21.ctumag
File not found: /Users/tristan/Library/CloudStorage/OneDrive-StellenboschUniversity/Academics/Final_year/Semester 2/Skripsie/Data/DUMMY/2024-03-21.squid
 Head of dataframe: 
                          NS_SQUID  F_SQUID  NS_Fluxgate  EW_Fluxgate  \
Time                                                                   
2024-03-21 15:15:10.540  -17.2443   0.3012   10934.2807     -42.4087   
2024-03-21 15:15:10.740  -17.2414   0.3024   10934.2699     -42.4171   
2024-03-21 15:15:10.940  -17.2352   0.2902   10934.2699     -42.4171   
2024-03-21 15:15:11.140  -17.2389   0.2985   10934.2653     -42.4182   
2024-03-21 15:15:11.340  -17.2338   0.2598   10934.2594     -42.3772   

                         Z_Fluxgate  
Time                                 
2024-03-21 15:15:10.540 -22656.4241  
2024-03-21 15:15:10.740 -22656.4286  
2024-03-21 15:

In [11]:
print(df.EW_Fluxgate.head())
print(df.EW_Fluxgate["2024-03-21 15:15:10.540":].resample("s").count())


Time
2024-03-21 15:15:10.540   -42.4087
2024-03-21 15:15:10.740   -42.4171
2024-03-21 15:15:10.940   -42.4171
2024-03-21 15:15:11.140   -42.4182
2024-03-21 15:15:11.340   -42.3772
Name: EW_Fluxgate, dtype: float64
Time
2024-03-21 15:15:10    3
2024-03-21 15:15:11    5
2024-03-21 15:15:12    5
2024-03-21 15:15:13    5
2024-03-21 15:15:14    5
                      ..
2024-03-22 17:32:35    4
2024-03-22 17:32:36    5
2024-03-22 17:32:37    5
2024-03-22 17:32:38    5
2024-03-22 17:32:39    5
Freq: s, Name: EW_Fluxgate, Length: 94650, dtype: int64


<Axes: xlabel='Time'>

Error in callback <function _draw_all_if_interactive at 0x1207d0900> (for post_execute), with arguments args (),kwargs {}:


KeyboardInterrupt: 

Plot the raw data

In [ ]:
eda.generateDataPlots(df['NS_SQUID'].values, df['F_SQUID'].values, df['NS_Fluxgate'].values, df['EW_Fluxgate'].values, df['Z_Fluxgate'].values, df.shape[0], 431997, start_date, end_date)

Check for missing values

In [ ]:
for column in df.columns:
    print(f"\n Number of missing values in {column} is: ", df[column].isna().sum())

Z score test

In [ ]:
print(f' \n Shape of df before removing outliers', df.shape)
removed_outliers_df = eda.z_score_test(df)
print(f' \n Shape of df after removing outliers', removed_outliers_df.shape)

Outlier check

In [ ]:
print(f' \n Shape of df before removing outliers', df.shape)
outliers_removed = eda.remove_outliers(df)
print(f' \n Shape of df after removing outliers', outliers_removed.shape)

Plot after outlier removed

In [ ]:
eda.plot_cleaned_data(outliers_removed)

Resampling

In [ ]:
outliers_removed.resample('s').count()


In [ ]:
resampled_df = eda.resample_data(outliers_removed, 's')

# Print the length of the resampled dataframe
print(f'This is the length of the resampled data frame', len(resampled_df))
# eda.generateDataPlots(resampled_df['NS_SQUID'].values, resampled_df['F_SQUID'].values, resampled_df['NS_Fluxgate'].values, resampled_df['EW_Fluxgate'].values, resampled_df['Z_Fluxgate'].values, resampled_df.shape[0], 300, start_date, end_date)
eda.plot_cleaned_data(resampled_df)
print(resampled_df)

Test for stationarity

In [ ]:
eda.perform_dickey_fuller_test(resampled_df)

Now perform differencing on data to ensure stationarity

In [ ]:
df_differenced = resampled_df.diff().dropna()
# print(f"This is the difference dataframe:\n",df_differenced)
eda.perform_dickey_fuller_test(resampled_df)

Test for seasonality and trend

In [ ]:
df = resampled_df
eda.test_stationarity(df)


Now using the autocorrelation_plot

In [ ]:
# Draw Plot
plt.rcParams.update({'figure.figsize':(9,5), 'figure.dpi':120})
autocorrelation_plot(df['EW_Fluxgate'].tolist())

Fourier Transform

In [ ]:
components, fourier_results = eda.calculate_fourier_transforms(df)
eda.plot_fourier_transform(fourier_results, components)

Write the preprocessed dataframe to a file

In [ ]:
df.to_csv('/Users/tristan/Library/CloudStorage/OneDrive-StellenboschUniversity/Academics/Final_year/Semester 2/Skripsie/Data/RESAMPLED/df.csv', index=True)

Create a pdf for each of the columns in the df

In [ ]:
import seaborn as sns

for column in df.columns:
    sns.displot(df[column], kde=True)

Decompose time series into its components

In [ ]:
# Additive Decomposition
# result_add = seasonal_decompose(df['Z_Fluxgate'], model='additive', extrapolate_trend='freq')

# Plot
# plt.rcParams.update({'figure.figsize': (10,10)})
# result_add.plot().suptitle('Additive Decompose', fontsize=22)
# plt.show()